In [1]:
import pandas as pd
# Geopandas is a geo dataframe
# import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# import cupy as cp
# print(cp.cuda.runtime.getDeviceCount())

# Load cleaned (preprocesses) data

In [3]:
all = pd.read_csv("/home/kim/data/bedmap/bedmap123_preprocessed.csv")
# might take 2 mins

In [4]:
# First two columns are unnessessary
all = all.iloc[:, 2:]

# Gridding function

In [5]:
def prune_all(df, res, min_count):
    zero_offset = res/2
    
    # grid at lowest res
    df["x_highestres"] = (np.round((df.x - zero_offset) / res) * res) + zero_offset  
    df["y_highestres"] = (np.round((df.y - zero_offset) / res) * res) + zero_offset  

    # Make categocial column based on grid cell index
    df["id_highestres"] = df["x_highestres"].astype(int).astype(str) + "_" + df["y_highestres"].astype(int).astype(str)

    # Drop above columns to avoid kernel crashing?!
    # df = df.drop(columns = [["x_highestres"], ["y_highestres"]])
    # print(df.shape)

    # b is arbitrary: just count rows
    grid_count_series = df.groupby(by = "id_highestres")["b"].count()
    # make dataframe
    grid_count_df = pd.DataFrame(grid_count_series)
    # rename from "b" to "grid_count"
    grid_count_df = grid_count_df.rename(columns = {"b": "grid_count"})

    df_count_merged = df.merge(grid_count_df, on = "id_highestres")
    print("Full dataframe:", df_count_merged.shape[0])

    df_pruned = df_count_merged.drop(df_count_merged.loc[(df_count_merged["grid_count"] < min_count)].index)
    print("Pruned dataframe:", df_pruned.shape[0])
    print("Percentage of measurements excluded due to count threshold:", np.round(df_pruned.shape[0]/df_count_merged.shape[0], 3), "with min_count", min_count, "and min resolution", res)

    return df_pruned

In [6]:
def gridding_error(df, zero_offset, res):
    # x grid values for 500m grid
    # 250 offset of 0 is first deducted
    df["x_500"] = (np.round((df.x - zero_offset) / res) * res) + zero_offset  
    df["y_500"] = (np.round((df.y - zero_offset) / res) * res) + zero_offset  

    # Make categocial column based on grid cell index
    df["id_500"] = df["x_500"].astype(int).astype(str) + "_" + df["y_500"].astype(int).astype(str)

    grid_b_series = df.groupby(by = "id_500")["b"].mean()
    grid_b_df = pd.DataFrame(grid_b_series)
    grid_b_df = grid_b_df.rename(columns = {"b": "grid_mean_b"})
    # count
    grid_count = df.groupby(by = "id_500")["b"].count()
    # add to df
    grid_b_df["count"] = np.array(grid_count)

    df_merged = df.merge(grid_b_df, on = "id_500")
    df_merged["error"] = df_merged["grid_mean_b"] - df_merged["b"]

    # Calculate rate of grid cells without measurements (in square)
    x_gridvalues = np.arange(np.min(df_merged.x_500), np.max(df_merged.x_500) + 1, res)
    y_gridvalues = np.arange(np.min(df_merged.y_500), np.max(df_merged.y_500) + 1, res)
    rate = grid_b_df.shape[0]/(x_gridvalues.shape[0] * y_gridvalues.shape[0])
      
    return  df_merged, rate

In [7]:
min_count = 2

# We now start with 62.5
res_list = [62.5, 125, 250]
# Initialise

rmse_list = []
mae_list = []
rate_list = []

# Smallest res as input
all_pruned = prune_all(all, res = res_list[0], min_count = min_count)

for i in res_list:
    zo = i/2
    res = i
    # run over all
    gridded_df, rate = gridding_error(all_pruned, zero_offset = zo, res = res)
    rmse = np.sqrt(np.mean(np.square(gridded_df[gridded_df["count"] > min_count]["error"])))
    print(rmse)
    mae = np.mean(np.abs(gridded_df[gridded_df["count"] > min_count]["error"]))
    # Append
    rmse_list.append(rmse)
    mae_list.append(mae)
    rate_list.append(rate)

    print("Resolution ", i, "completed.")

Full dataframe: 65238004
Pruned dataframe: 61229863
Percentage of measurements excluded due to count threshold: 0.939 with min_count 2 and min resolution 62.5


: 

In [ ]:
min_count = 2

# We now start with 62.5
res_list2 = [500, 1000, 2000]
# Initialise

rmse_list2 = []
mae_list2 = []
rate_list2 = []

# Smallest res as input
# all_pruned = prune_all(all, res = res_list[0], min_count = min_count)

for i in res_list2:
    zo = i/2
    res = i
    # run over all
    gridded_df, rate = gridding_error(all_pruned, zero_offset = zo, res = res)
    rmse = np.sqrt(np.mean(np.square(gridded_df[gridded_df["count"] > min_count]["error"])))
    print(rmse)
    mae = np.mean(np.abs(gridded_df[gridded_df["count"] > min_count]["error"]))
    # Append
    rmse_list2.append(rmse)
    mae_list2.append(mae)
    rate_list2.append(rate)

    print("Resolution ", i, "completed.")

Run in 2 portions or otherwise import cupy as np

In [ ]:
res_list_combined = np.concatenate((res_list, res_list2))
rmse_list_combined = np.concatenate((rmse_list, rmse_list2))
mae_list_combined = np.concatenate((mae_list, mae_list2))
rate_list_combined = np.concatenate((rate_list, rate_list2))

# Save results

In [ ]:
data = {'resolution': res_list,
        'rmse': rmse_list,
        'mae': mae_list,
        'rate': rate_list}

experiment_results_all = pd.DataFrame(data)

experiment_results_all.to_csv('results/experiment_results_all.csv', index=False)

## Plots

In [ ]:
plt.plot(res_list, rmse_list, "--bo")

plt.xlabel("grid cell size (resolution)")
plt.ylabel("RMSE")
plt.ylim(bottom = 0)
plt.title("Gridding error (RMSE) by resolution for Antarctica (61 M measurements)")
plt.axvline(x = 500, ls='--', color = "lightgrey")

In [ ]:
plt.plot(res_list, mae_list, "--bo")

plt.xlabel("grid cell size (resolution)")
plt.ylabel("MAE")
plt.title("Gridding error (MAE) by resolution for Antarctica (61 M measurements)")
plt.ylim(bottom = 0)
plt.axvline(x = 500, ls='--', color = "lightgrey")

In [ ]:
gridded_df.sort_values('error')

## Dome A

In [ ]:
domea = all[(all["x"] > 898750) & (all["x"] < 1498750) & (all["y"] > 250) & (all["y"] < 600250)]
# 1.8 M values in dome A region

# The domain lies between [898,750, 1,498,750] projection
# x coordinate (in meters), and [250, 600,250] projection y coordinate (in meters).

np.mean(domea["t"])

In [ ]:
domea.to_csv("/home/kim/data/bedmap/bedmap123_preprocessed_DomeA.csv")

In [ ]:
plt.plot(res_list, da_rmse_list, "--bo", label = "Dome A")
plt.plot(res_list, rmse_list, "--go", label = "Antarctica")
plt.xlabel("grid resolution")
plt.ylabel("RMSE")
plt.title("Gridding error (RMSE) by resolution")
plt.legend()
plt.ylim(bottom = 0)
plt.axvline(x = 500, ls='--', color = "lightgrey")
plt.show()

In [ ]:
plt.plot(res_list, da_mae_list, "--bo", label = "Dome A")
plt.plot(res_list, mae_list, "--go", label = "Antarctica")
plt.xlabel("grid resolution")
plt.ylabel("MAE")
plt.title("Gridding error (MAE) by resolution")
plt.legend()
plt.ylim(bottom = 0)
plt.axvline(x = 500, ls='--', color = "lightgrey")
plt.show()